In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split, KFold
# save the final model to file
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model

In [9]:
def normalize(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm
def loadData():
    # listImagePaths=glob('characterdata//0/*.jpg')
    listLabel=os.listdir('charTrainset//')
    listImages=[]
    labels=[]
	# label for number train
    # maplabel={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9}
	# label for character train
    maplabel={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'P':14,'Q':15,'R':16,'S':17,'T':18,'U':19,'V':20,'W':21,'X':22,'Y':23,'Z':24}
    for label in maplabel.keys():
        listImagePaths=glob('charTrainset//'+label+'/*.jpg')
        len_listImagePath=len(listImagePaths)
        for i in range(len_listImagePath):
            img=~cv2.imread(listImagePaths[i], 0)
            # img=cv2.resize(img,(30,40))
            # extend=np.zeros((40,5))
            # img=np.hstack((extend, img))
            # img=np.hstack((img, extend))
            img=cv2.resize(img, (30,30))
            labels.append(maplabel[label])
            listImages.append(img)
    listImages=np.asarray(listImages)
    labels=np.asarray(labels)
    listImages=listImages.reshape(listImages.shape[0], 30,30,1)
    X_train, X_test, y_train, y_test = train_test_split(listImages, labels, test_size=0.2)
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    # normalize
    return (X_train, X_test, y_train, y_test)
def imshow(img):
    plt.figure(figsize=(6,6))
    plt.imshow(img, cmap='gray')
    plt.show()
# define cnn model

def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(30, 30, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(25, activation='sigmoid'))
	# compile model
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
	# model.summary()
	return model
# evaluate a model using k-fold cross-validation
def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	# prepare cross validation
	i=0
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	# enumerate splits
	for train_ix, test_ix in kfold.split(dataX):
		# define model
		model = define_model()
		# select rows for train and test
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		# fit model
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		model.save('model_character'+str(i)+'.h5')
		i=i+1
		# evaluate model
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		# stores scores
		scores.append(acc)
		histories.append(history)
	return scores, histories
# plot diagnostic learning curves
def summarize_diagnostics(histories):
	for i in range(len(histories)):
		# plot loss
		pyplot.subplot(3, 1, 1)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
		# plot accuracy
		pyplot.subplot(3, 1, 3)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX,testX, trainY, testY = loadData()
	# prepare pixel data
	trainX, testX = normalize(trainX, testX)
	# define model
	# scores, histories = evaluate_model(trainX, trainY)
	# model=define_model()
	# #fit model
	# # print(trainY.shape)
	# # print(trainX.shape)
	# model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)
	# # save model
	# model.save('number_model.h5')

In [10]:
trainX,testX, trainY, testY = loadData()
# prepare pixel data
trainX, testX = normalize(trainX, testX)
# define model
# scores, histories = evaluate_model(trainX, trainY)

In [17]:
maplabel={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,'N':13,'P':14,'Q':15,'R':16,'S':17,'T':18,'U':19,'V':20,'W':21,'X':22,'Y':23,'Z':24}
inv_map = {v: k for k, v in maplabel.items()}
# model=load_model('model_character1.h5')
# _, acc=model.evaluate(testX, testY, verbose=0)
print("%.3f" %(acc*100.0))
for i in testX[:24]:
    plt.imshow(i, cmap='gray')
    plt.show()
    model=load_model('model_character3.h5')
    digit=model.predict_classes(i.reshape(1,30,30,1))
    print(inv_map[digit[0]])

NameError: name 'acc' is not defined

In [13]:
import matplotlib.pyplot as pyplot
summarize_diagnostics(histories)

NameError: name 'histories' is not defined

In [15]:
model=load_model('model_character1.h5')
_, acc=model.evaluate(testX, testY, verbose=0)
print("%.3f" %(acc*100.0))

UnicodeEncodeError: 'mbcs' codec can't encode characters in position 0--1: invalid character